# Constraining Gradients

In this demonstration, we show how you can expose a bit more of the automatic differentiation engine provided by CasADi. Credit to Antonios Gementzopoulos (@antoniosgeme) for working out bugs with the notation here.

One use case for this might be constraining a gradient of a function; for example:

* For an aircraft design problem, you might wish to add a static stability constraint of the form $d(Cm)/d(\alpha) < -1$ or similar. You could directly compute $d(Cm)/d(\alpha)$ (more commonly abbreviated $C_{m \alpha}$) and add a constraint on that.

For a simple example of this, we show how you can solve the following optimization problem:

* Where $f(x) = x^4$:
* Minimize $0$ (in other words, there is no objective - simply find any feasible solution)
* Subject to:
    * $df/dx = 1$

We can analytically compute the solution:

$df/dx = 4x^3 = 1 \implies x = (1/4)^{1/3} \approx 0.629961$

We set up the problem using familiar syntax:

In [1]:
import aerosandbox as asb

opti = asb.Opti()

x = opti.variable(init_guess=10)


def f(x):
    return x**4

So far, this is all pretty normal. Now, we compute the gradient using a bit of CasADi magic:

In [2]:
import casadi as cas  # Import CasADi

dummy = cas.GenMX_sym(
    "s"
)  # Create a dummy variable aliased to "s" (name is not important, only useful for debugging)

dfdx = cas.Function(  # Create a Python function...
    "dfdx",  # Aliased to "dfdx" (name is not important, only useful for debugging).
    [dummy],  # Input to the function should be some dummy variable.
    [
        cas.gradient(  # Output of the function should be the gradient of...
            f(dummy),  # The function applied to our dummy variable...
            dummy,  # With respect to the dummy variable.
        )
    ],
)

And now we add our constraint and solve, turning verbosity off just to keep things a bit cleaner.

In [3]:
opti.subject_to(dfdx(x) == 1)

sol = opti.solve(verbose=False)

print(f"x_optimal = {sol(x)}")

x_optimal = 0.6299605894866889


Which we can then compare to our analytical solution:

In [4]:
print((1 / 4) ** (1 / 3))

0.6299605249474366


So, it worked!